# WebScrapping, Web automation using Julia Langauge

## AI Bot - automating document download using Chrome, Selenium webdriver

**Objective** This AI bot, take print screens, download documents from web browser.
AI Script, uses **headless chrome webdriver** for proxy browsing.

Further, GUMBO and Cascadia packages are used to crawl and read web contents.

- Related Article: how to parse XML/JSON files and convert into ORM
- Related Article: how to Read Scanned Images and parse PDF files

---

+ First step is to [download chrome webdriver](https://chromedriver.chromium.org/downloads). Please make sure, webdriver version matches with your chrome version (Open Chrome -> Help -> About chrome -> check version).
+ make sure to download appropriate version depending on machine OS and unzip/extract to a local folder.
+ Open a terminal window, browse to directory where webdriver is downloaded and start webdriver as ./webdriver --url-base=/wd/hub

following message should appear on your terminal window.

In [1]:
import Pkg
Pkg.add("WebDriver")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
using WebDriver
capabilities = Capabilities("chrome")
ENV["WEBDRIVER_HOST"] = "127.0.0.1"
ENV["WEBDRIVER_PORT"] = "9515"

"9515"

In [3]:
wd = RemoteWebDriver(capabilities, host=ENV["WEBDRIVER_HOST"], port=parse(Int, ENV["WEBDRIVER_PORT"]))

Remote WebDriver


In [18]:
session = Session(wd)

Session


In [6]:
sessions(wd)

1-element Vector{String}:
 "f00406b9c4696534906c90d93e9bf492"

In [5]:
current_url(session)

"data:,"

In [7]:
navigate!(session, "https://clinicaltrials.gov")

once, website is open, in your chrome browser window, right click on Input or Button you want to search or click on.
make sure, element is selected then click on copy -> copy xpath

this is the xpath you want to pass in your Element method.

    //*[@id="home-search-other-query"]

In [8]:
element = Element(session, "xpath", """//*[@id="home-search-other-query"]""")
element_keys!(element, "Cardiac Arrest")

In [10]:
downloadbtn = Element(session, "xpath", """//*[@id="save-list-link"]""")
click!(downloadbtn)

In [11]:
savebtn = Element(session, "xpath", """//*[@id="submit-download-list"]""")
click!(savebtn)

## Taking screen-shot and saving as png file

In [12]:
using Base64
ss = write(joinpath(@__DIR__, "my_screenshot.png"), base64decode(screenshot(session)))

108173

## Webscrapping


Gumbo.jl is a Julia wrapper around Google's gumbo library for parsing HTML 

and Cascadia.jl package is used for automatic extraction of information from html pages.

In [13]:
Pkg.add("HTTP")
Pkg.add("Gumbo")
Pkg.add("Cascadia")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
   Installed Cascadia ─ v1.0.1
    Updating `~/.julia/environments/v1.7/Project.toml`
  [54eefc05] + Cascadia v1.0.1
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [54eefc05] + Cascadia v1.0.1
Precompiling project...
  ✓ Cascadia
  1 dependency successfully precompiled in 0 seconds (76 already precompiled)


## Download data from web page

In [35]:
# download contact details from my GitHub page
navigate!(session, "https://github.com/AmitXShukla")
html = parsehtml(source(session))
eachmatch(Selector("#js-pjax-container"), html.root)

1-element Vector{HTMLNode}:
 HTMLElement{:main}:<main data-pjax-container="" id="js-pjax-container">
  <div class="mt-4 position-sticky top-0 d-none d-md-block color-bg-default width-full border-bottom
    color-border-muted" style="z-index:3;">
    <div class="container-xl px-3 px-md-4 px-lg-5">
      <div class="Layout Layout--flowRow-until-md Layout--sidebarPosition-start Layout--sidebarPosition-flowRow-start" data-view-component="true">
        <div class="Layout-sidebar" data-view-component="true">
          <div class="user-profile-sticky-bar">
            <div class="user-profile-mini-vcard d-table">
              <span class="user-profile-mini-avatar d-table-cell v-align-middle lh-condensed-ultra pr-2">
                <img alt="@AmitXShukla" class="rounded-2 avatar-user" height="32" src="https://avatars.githubusercontent.com/u/20031132?s=64&amp;v=4" width="32"/>
              </span>
              <span class="d-table-cell v-align-middle lh-condensed">
                <strong>

In [16]:
using Cascadia, Gumbo, HTTP

r = HTTP.get("http://stackoverflow.com/questions/tagged/julia-lang")
h = parsehtml(String(r.body))

qs = eachmatch(Selector(".question-summary"),h.root)

println("StackOverflow Julia Questions (votes  answered?  url)")

for q in qs
    votes = nodeText(eachmatch(Selector(".votes .vote-count-post "), q)[1])
    answered = length(eachmatch(Selector(".status.answered"), q)) > 0 || length(eachmatch(Selector(".status.answered-accepted"), q)) > 0
    href = eachmatch(Selector(".question-hyperlink"), q)[1].attributes["href"]
    votes, answered, "http://stackoverflow.com",href
end

StackOverflow Julia Questions (votes  answered?  url)
